In [1]:
from osgeo import gdal ,gdal_array,ogr ,osr
import numpy as np 
import os 


In [40]:

# Define raster and shapefile paths
rstr_pth = rf'/home/mahmoud/Produced_Wetland_Rasters/Oksbotn.tif'
gdp_pth  = rf'/home/mahmoud/Mahmoud_Saber_Kenawi/WETLAND/Reservoir_SHP/Analyzed_Res.shp'
gdb = ogr.Open(gdp_pth)
rstr = gdal.Open(rstr_pth)
base_name = os.path.splitext(os.path.basename(rstr_pth))[0]
layer = gdb.GetLayerByIndex(0)
raster_gt = rstr.GetGeoTransform()
raster_proj = rstr.GetProjection()

# Calculate the raster's bounds (extent)
raster_minx = raster_gt[0]
raster_maxx = raster_gt[0] + raster_gt[1] * rstr.RasterXSize
raster_miny = raster_gt[3] + raster_gt[5] * rstr.RasterYSize
raster_maxy = raster_gt[3]
# Define raster bounding box
raster_extent = (raster_minx, raster_maxy, raster_maxx, raster_miny)

output_dir = rf'/home/mahmoud/Reservoir_Wetland_Files'
options = ["COMPRESS=LZW"]

# Prepare a spatial reference object from the raster
raster_srs = osr.SpatialReference()
raster_srs.ImportFromWkt(rstr.GetProjection())

# Set a spatial filter on the shapefile layer to only include features that overlap the raster extent
layer.SetSpatialFilterRect(raster_minx, raster_miny, raster_maxx, raster_maxy)

In [41]:
# Define output shapefile directory and temporary shapefile path
shapefile_output_dir = rf'/home/mahmoud/Reservoir_Wetland_Files/Shapefiles'
output_shp_path = os.path.join(shapefile_output_dir, 'temp_feature.shp')  # Single shapefile that will be overwritten

# Ensure output directory exists
if not os.path.exists(shapefile_output_dir):
    os.makedirs(shapefile_output_dir)

# Loop over each feature in the layer
for feature in layer:
    fid = feature.GetFID()  # Use the Feature ID (FID) to track each feature
    Sch_name = feature.GetField('Scheme_Nam')
    Mag_Name = feature.GetField("magasinNav")
    Mag_Type = feature.GetField("Reservoir_")
    Mag_nr = feature.GetField('magasinNr')
    HPnr = feature.GetField('vannkraftv')
    
    # Condition to skip processing if Mag_Type is not "Built" or "Expand"
    if Mag_Type not in ["Built", "Expand"]:
        print(f"Skipping feature {fid} with Mag_Type '{Mag_Type}'")
        continue

    # Create or overwrite the shapefile for the individual feature
    driver = ogr.GetDriverByName('ESRI Shapefile')
    out_ds = driver.CreateDataSource(output_shp_path)
    if out_ds is None:
        print(f"Could not create shapefile {output_shp_path}")
        continue

    # Create a new layer with the same spatial reference as the original
    out_layer = out_ds.CreateLayer('feature_layer', geom_type=layer.GetGeomType(), srs=raster_srs)

    # Add the feature's geometry to the shapefile
    out_feature = ogr.Feature(out_layer.GetLayerDefn())
    out_feature.SetGeometry(feature.GetGeometryRef().Clone())
    out_layer.CreateFeature(out_feature)

    # Clean up the feature and datasource
    out_feature = None
    out_ds = None

    # Define the output raster path for each cut
    output_raster_path = os.path.join(output_dir, f'{Mag_Name}_nr_{Mag_nr}_MagType_{Mag_Type}_HPnr_{HPnr}.tif')

    # Use gdal.Warp to create a new raster based on the cutline from the shapefile
    gdal.Warp(output_raster_path, rstr_pth, cutlineDSName=output_shp_path, cropToCutline=True, creationOptions=['COMPRESS=LZW','TILED=YES','NUM_THREADS=ALL_CPUS'],dstNodata = 0)

    print(f"Feature {fid} processed and saved as raster {output_raster_path}")

# Reset the spatial filter after looping
layer.SetSpatialFilter(None)


Feature 30 processed and saved as raster /home/mahmoud/Reservoir_Wetland_Files/Nedsta Piksvatnet_nr_689_MagType_Expand_HPnr_554.tif
Feature 31 processed and saved as raster /home/mahmoud/Reservoir_Wetland_Files/Stora Volavatnet_nr_688_MagType_Expand_HPnr_554.tif


In [5]:
layer = gdb.GetLayerByIndex(0)  
layer_name = layer.GetName()
layer_defn = layer.GetLayerDefn()
# Print fields in the layer
print(f"Fields in Layer '{layer_name}':")
for i in range(layer_defn.GetFieldCount()):
    field_defn = layer_defn.GetFieldDefn(i)
    print(f"{i}: {field_defn.GetName()} ({field_defn.GetTypeName()})")

Fields in Layer 'Analyzed_Res':
0: objektType (String)
1: vatnLnr (Integer64)
2: magasinNr (Integer)
3: magasinNav (String)
4: magasinKat (Integer)
5: lavesteReg (Real)
6: hoyesteReg (Real)
7: status (String)
8: idriftsatt (Integer)
9: kdbNr (Integer)
10: konsesjonS (Integer)
11: konsesjo_1 (Date)
12: samletPlan (String)
13: magasinFor (String)
14: magasinAre (Real)
15: volumOppde (Real)
16: delfeltNr (Integer)
17: vannkraftv (Integer)
18: vannkraf_1 (String)
19: vassdragsn (String)
20: elvenavnHi (String)
21: oppdaterin (Date)
22: dataUttaks (Date)
23: eksportTyp (String)
24: Buf (Real)
25: Buf2 (Real)
26: Area_Km2 (Real)
27: OBJECTID (Integer64)
28: Scheme_Nam (String)
29: Hp_Name (String)
30: COL_C (String)
31: HP_Nr (String)
32: idriftAar (String)
33: maksYtelse (String)
34: fylke (String)
35: magasinNr_ (String)
36: magasinN_1 (String)
37: Res (Integer64)
38: INTKs (Integer64)
39: Area_Res (Real)
40: Res_Date (Integer64)
41: Reservoir_ (String)
42: Data_from (String)
43: T1_Proces